In [1]:
from __future__ import division

import pandas as pd
import numpy as np
#import html2text
from datetime import date
import random
import os

#Packages for pulling text data 
from urllib.request import urlopen  # the lib that handles the url stuff
from bs4 import BeautifulSoup
import pandas.io.data as web
from pandas.tseries.offsets import BDay

#Packages for text data processing
import nltk, re, pprint
#nltk.download()
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer

#Packages for statistical learning 
from sklearn.preprocessing import normalize as Normal
from sklearn import svm
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
from sklearn import linear_model

import matplotlib.pyplot as plt
%matplotlib inline

import imp

import Text_Analytics as TA

C:\Users\ziyi\Anaconda3\lib\site-packages\pandas\io\data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [2]:
#from imp import reload 
imp.reload(TA)

<module 'Text_Analytics' from 'C:\\Users\\ziyi\\Desktop\\Inde project\\Independent Study Anseri-20170120T004329Z\\Independent Study Anseri\\Code repo\\Text_Analytics.py'>

In [3]:
#Setup working directory 
os.chdir("C:/Users/ziyi/Desktop/Inde project/Independent Study Anseri-20170120T004329Z/Independent Study Anseri")

#Import Data
df=pd.read_excel('8K data/8K_Info.xlsx')
Index=pd.read_csv("8K data/Index_Info.csv",index_col=0)

#Time Range under study
Time_range=pd.date_range("2013-01-01","2016-12-31",freq="D")

In [4]:
Index=pd.read_csv("8K data/Index_Info.csv",index_col=0)
Index.head()

,Ticker,Name,Market Cap,Weight
8,MPWR,MONOLITHIC POWER SYSTEMS INC,3462565376,0.011873
9,BDC,BELDEN INC,3087077888,0.010586
10,ACXM,ACXIOM CORP,2208818432,0.007574
13,WEX,WEX INC,4759351296,0.016320
15,PAY,VERIFONE SYSTEMS INC,2156781056,0.007396


In [5]:
df.head()

,Unnamed: 0,CIK,Company Name,Form Type,Date Filed,Filename,Text,Ticker
9,3732,1009626,BROCADE COMMUNICATIONS SYSTEMS INC,8-K,1/7/2015,edgar/data/1009626/0001009626-15-000002.txt,ITEM 5.02DEPARTURE OF DIRECTORS OR CERTAIN OFF...,BRCD
10,3733,1009626,BROCADE COMMUNICATIONS SYSTEMS INC,8-K,1/7/2015,edgar/data/1009626/0001193125-15-004052.txt,"ITEM 8.01\nOTHER EVENTS. ON JANUARY 7, 2015, ...",BRCD
11,3734,1009626,BROCADE COMMUNICATIONS SYSTEMS INC,8-K,1/14/2015,edgar/data/1009626/0001193125-15-010610.txt,ITEM 1.01\nENTRY INTO A MATERIAL DEFINITIVE AG...,BRCD
12,3735,1009626,BROCADE COMMUNICATIONS SYSTEMS INC,8-K,1/30/2015,edgar/data/1009626/0001009626-15-000010.txt,ITEM 5.02DEPARTURE OF DIRECTORS OR CERTAIN OFF...,BRCD
13,3736,1009626,BROCADE COMMUNICATIONS SYSTEMS INC,8-K,2/19/2015,edgar/data/1009626/0001009626-15-000013.txt,ITEM 2.02 RESULTS OF OPERATIONS AND FINANCIAL ...,BRCD


In [6]:
events_list=("ITEM 1.01","ITEM 1.02","ITEM 1.03","ITEM 1.04",
             "ITEM 2.01","ITEM 2.02","ITEM 2.03","ITEM 2.04","ITEM 2.05","ITEM 2.06",
             "ITEM 3.01","ITEM 3.02","ITEM 3.03",
             "ITEM 4.01","ITEM 4.02",
             "ITEM 5.01","ITEM 5.02","ITEM 5.03","ITEM 5.04","ITEM 5.05","ITEM 5.06","ITEM 5.07","ITEM 5.08",
             "ITEM 6.01","ITEM 6.02","ITEM 6.03","ITEM 6.04","ITEM 6.05",
             "ITEM 7.01",
             "ITEM 8.01",
             "ITEM 9.01"            
            )

In [7]:
Events_df=TA.Doc_Events(df,events_list)
#Events_df.to_csv('8K data/events_Info.csv')

Events_df_new=Events_df.merge(Index[["Name","Weight"]],right_on="Name",left_on="Company Name",how="left")
Events_df_new=Events_df_new.drop("Name",axis=1)
Events_df_new2=Events_df_new.copy()
Events_df_new2=Events_df_new2.drop("Weight",axis=1)

for item in Events_df_new2.columns[3:]:
    Events_df_new2[item]=Events_df_new[item]*Events_df_new["Weight"]

Events_df_new2.head()

In [8]:
EventsbyComp=Events_df.groupby("Company Name").sum()
EventsbyComp=EventsbyComp.drop("CIK",1)
print(EventsbyComp.sum(axis=1))
print(EventsbyComp.sum(axis=0))

Company Name
3D SYSTEMS CORP                       194.0
ACXIOM CORP                            68.0
ADVANCED MICRO DEVICES INC            123.0
ANSYS INC                              57.0
ARROW ELECTRONICS INC                  86.0
AVNET INC                              67.0
BROCADE COMMUNICATIONS SYSTEMS INC     95.0
CADENCE DESIGN SYSTEMS INC             60.0
CIENA CORP                             77.0
CIRRUS LOGIC INC                      100.0
COGNEX CORP                            60.0
COMMVAULT SYSTEMS INC                  43.0
COMPUTER SCIENCES CORP                182.0
CONVERGYS CORP                         70.0
CREE INC                               91.0
DST SYSTEMS INC                       108.0
FAIR ISAAC CORP                        48.0
FORTINET INC                           67.0
GARTNER INC                            68.0
INTEGRATED DEVICE TECHNOLOGY INC       81.0
IPG PHOTONICS CORP                     88.0
JABIL CIRCUIT INC                      50.0
MANHATTAN ASSOCIATE

In [9]:
Events_ts=TA.Index_Events(Events_df,Index,Time_range)
Events_ts.head()

,ITEM 1.01,ITEM 1.02,ITEM 1.03,ITEM 1.04,ITEM 2.01,ITEM 2.02,ITEM 2.03,ITEM 2.04,ITEM 2.05,ITEM 2.06,...,ITEM 5.07,ITEM 5.08,ITEM 6.01,ITEM 6.02,ITEM 6.03,ITEM 6.04,ITEM 6.05,ITEM 7.01,ITEM 8.01,ITEM 9.01
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2013-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.042144
2013-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006743,0.006743
2013-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2013-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [10]:
Events_ts.to_csv('8K data/events_ts_Info.csv')